In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objs as go
import plotly.express as px

import warnings
warnings.filterwarnings("ignore")

El gobierno de EEUU quiere que las multas a la aerolíneas sean directamente proporcionales a la distancia del trayecto realizado, entendemos que un trayecto corto es aquel que tiene una distancia inferior a 750, uno medio tiene una distancia comprendida entre 750 y 1500 y uno largo es aquel que supera las 1500 millas. Si un vuelo se llega antes de media hora más tarde respecto al horario previsto, asumiremos que ese retraso no ha tenido impacto, si se retrasa entre media hora y una hora se debera devolver el 50% de los billetes a los pasajeros y una multa den función de la distancia. Así mismo si el retraso es superior a 1 hora se deberá devolver el importe total de los billetes a los pasajeros y se deberá pagar una multa mayor que la del caso anterior.

**NOTA** Las aerolíneas se consideran plenamente responsables del retraso de los vuelos independientemente de la causa

In [ ]:
# Cargamos los datos y seleccionamos las columnas
df = pd.read_parquet("../Preprocessing/flightsCleaned.parquet")
#df.head()

In [ ]:
#df.columns

In [ ]:
# Seleccionamos las variables que nos interesan para el análisis de las multas
fines_df = df.loc[:, ['DATE','AIRLINE', 'FLIGHT_NUMBER','ARRIVAL_DELAY','AIRLINE_DELAY','DISTANCE']]
#fines_df.head()

En primer lugar vamos a analizar el tráfico aéreo por aerolínea

In [ ]:
airlines = pd.DataFrame(fines_df.groupby(fines_df['AIRLINE'])["FLIGHT_NUMBER"].count())
airlines = airlines.rename(columns={"FLIGHT_NUMBER":"TOTAL_FLIGHTS"})
airlines["DELAYED_FLIGHTS"] =  fines_df[fines_df["ARRIVAL_DELAY"]>0].groupby(fines_df['AIRLINE'])["FLIGHT_NUMBER"].count()
airlines["AIRLINE_DELAYED_FLIGHTS"] =  fines_df[fines_df["AIRLINE_DELAY"]>0].groupby(fines_df['AIRLINE'])["FLIGHT_NUMBER"].count()
airlines = airlines.sort_values('TOTAL_FLIGHTS',ascending=False)
airlines = airlines.reset_index(level=0, drop=False)


fig = go.Figure()

fig.add_trace(go.Bar(
    x=airlines['AIRLINE'],
    y=airlines['TOTAL_FLIGHTS'],
    name='Total Flights',
    marker_color=px.colors.qualitative.Vivid[5]
))
fig.add_trace(go.Bar(
    x=airlines['AIRLINE'],
    y=airlines['DELAYED_FLIGHTS'],
    name='Delayed Flights',
    marker_color=px.colors.qualitative.Vivid[9]
))
fig.add_trace(go.Bar(
    x=airlines['AIRLINE'],
    y=airlines['AIRLINE_DELAYED_FLIGHTS'],
    name='Airline Delayed Flights',
    marker_color=px.colors.qualitative.Vivid[10]
))

# Here we modify the tickangle of the xaxis, resulting in rotated labels.
fig.update_layout(
    title="Flights Distribution per Airline",
    xaxis_title="Airlines",
    yaxis_title="Air traffic",
    legend_title="Leyend",
    template="plotly_dark",
    barmode='group', 
    xaxis_tickangle=-45   
)

fig.update_layout(barmode='group', xaxis_tickangle=-45, height = 650)
fig.show()

A continuación creamos una serie de variables relacionadas con las rutas y las multas

In [ ]:
# Creamos la columna tipo de vuelos según la distancia
condition_distance = [
    (fines_df['DISTANCE'] < 750) ,
    (fines_df['DISTANCE'] >= 750) & (fines_df['DISTANCE'] <1500),
    (fines_df['DISTANCE'] >= 1500)]

choice_distance = ['Short', 'Mid', 'Long']

# Creamos la columna tipo de retraso
condition_delay = [
    (fines_df['ARRIVAL_DELAY'] <= 0) ,
    (fines_df['ARRIVAL_DELAY'] > 0) & (fines_df['ARRIVAL_DELAY'] <= 30),
    (fines_df['ARRIVAL_DELAY'] > 30) & (fines_df['ARRIVAL_DELAY'] <=60),
    (fines_df['ARRIVAL_DELAY'] > 60)]

choice_delay = ['Early arrival', '(0-30)mins', '(30-60)mins','>1h']

# Añadimos las columnas
fines_df['DISTANCE_TYPE'] = np.select(condition_distance, choice_distance, default='Not Specified')
fines_df['DELAY_TYPE'] = np.select(condition_delay, choice_delay, default='Not Specified')

In [ ]:
# Agrupamos por aerolineas la info de los retrasos
level_count = pd.DataFrame(fines_df.groupby("AIRLINE")["DELAY_TYPE"].value_counts()).rename(columns = {"DELAY_TYPE": "count"}).reset_index()
level_count = level_count.sort_values('count',ascending=False)

fig = px.histogram(level_count, x="AIRLINE", y="count",barnorm='percent', color="DELAY_TYPE",text_auto='.2f',
                   title="Flights Distribution per Airline", color_discrete_sequence=px.colors.qualitative.Vivid, template="plotly_dark")

fig.update_layout(
    title="Flights Distribution per Airline",
    xaxis_title="Airlines",
    yaxis_title="% of flights per delay type",
    legend_title="Leyend",
    template="plotly_dark",
    hovermode="x unified", 
    height = 650
)


fig.show()

El 80% de los vuelos de todas las aerolíneas están exentos de multa dado el tipo de retraso. No obstante, Frontier y Spririt destacan por ser las dos compañías con menor % de vuelos libres de multas. Vamos a analizar más detenidamente los vuelos objetivo de multa

In [ ]:
# Agrupamos por aerolineas la info de los retrasos
level_count = pd.DataFrame(fines_df[fines_df["AIRLINE_DELAY"]>0].groupby("AIRLINE")["DELAY_TYPE"].value_counts()).rename(columns = {"DELAY_TYPE": "count"}).reset_index()
level_count = level_count.loc[(level_count['DELAY_TYPE'] == '(30-60)mins') | (level_count['DELAY_TYPE'] == ">1h")]
level_count = level_count.sort_values('count',ascending=False)

fig = px.histogram(level_count, x="AIRLINE", y="count", color="DELAY_TYPE",text_auto='.f',
                   title="Flights Distribution per Airline", color_discrete_sequence=px.colors.qualitative.Vivid[2:], template="plotly_dark")

fig.update_layout(
    title="Late arrival flights distribution per Airline",
    xaxis_title="Airlines",
    yaxis_title="Amount of flights per delay type",
    legend_title="Leyend",
    template="plotly_dark",
    hovermode="x unified", 
    height = 650
)


fig.show()

Alarmante el caso de Southwest. En la gráfica anterior veíamos que en torno al 89% de sus vuelos llegaban a destino con un máximo de retraso de 30 min. En esta gráfica se observa que ese 11% restante incluye una gran cantidad de vuelos, lo que podría costarle a la compaía mucho dinero.

# FINES

Vamos a analizar cuánto le costaría a cada compañia dada esta distribución de retrasos

**RECAP:** Las compañias pagarán una multa en función de la distancia del trayecto y el tiempo de retraso. Además también deberén reembolsar a los pasajeros un % del importe de su billete como consecuencia del tiempo de retraso

Para calcular cuánto se deberá pagar en cada caso necesitamos hacer una serie de estimaciones las cuales detallaremos a continuación.

**IMPORTE DEL BILLETE**

"In 2015, an average flight cost $430." https://www.mercurynews.com/2016/07/08/see-how-the-cost-of-a-flight-has-changed-since-1963/

Asociamos avg flight con distancia media. También asumimos que a mayor distancia mayor precio (un 30% superior al avg), así mismo como a menor distancia, menor precio (un 30% inferior al avg). Como consecuencia, asumimos que el importe de cada billete en función de la distancia es el siguiente:

In [ ]:
AVG_TICKET_SHORT = 0.7*430 # $301
AVG_TICKET_MID = 430
AVG_TICKET_LONG = 1.3*430  # $559

**NIVEL DE OCUPACIÓN**

Por otro lado, necesitamos estimar el número de pasajeros por vuelo para estimar el importe a pagar a los pasajeros en los vuelos que sufren mayores retrasos. De acuerdo con https://datos.bancomundial.org/indicator/IS.AIR.PSGR?locations=US en el año 2015 se transportaron en Estados Unidos un total de 798,222,000 pasajeros en todo el año. Asumimos una distribución uniforme de pasajeros en los vuelos dado que no existen evidencias claras de que la capacidad de los aviones sea fija en función de la distancia.

Dicho esto:

In [ ]:
total_passsengers = 798222000
total_flights = len(fines_df)
pass_per_flight = round(total_passsengers/total_flights,2) # 139,76
pass_per_flight = int(round(pass_per_flight,0))
pass_per_flight

#### Cálculo del importe a pagar 

In [ ]:
# Retrasos (30-60) mins --> multa en función de la distancia + 50% del importe del billete
AVG_TICKET_SHORT_TYPE_I =  5000 + 0.5*pass_per_flight*AVG_TICKET_SHORT
AVG_TICKET_MID_TYPE_I   = 10000 + 0.5*pass_per_flight*AVG_TICKET_MID
AVG_TICKET_LONG_TYPE_I  = 20000 + 0.5*pass_per_flight*AVG_TICKET_LONG

# Retrasos >1h --> multa en función de la distancia + 100% del importe del billete
AVG_TICKET_SHORT_TYPE_II =  7500 + 0.5*pass_per_flight*AVG_TICKET_SHORT
AVG_TICKET_MID_TYPE_II   = 20000 + 0.5*pass_per_flight*AVG_TICKET_MID
AVG_TICKET_LONG_TYPE_II  = 40000 + 0.5*pass_per_flight*AVG_TICKET_LONG

# Creamos una lista de los $ a pagar en función de la distancia y el retraso de los vuelos
choice_multa = [0,AVG_TICKET_SHORT_TYPE_I,AVG_TICKET_MID_TYPE_I,AVG_TICKET_LONG_TYPE_I,
                AVG_TICKET_SHORT_TYPE_II,AVG_TICKET_MID_TYPE_II,AVG_TICKET_LONG_TYPE_II]

# Creamos la columna correspondiente a la multa
condition_multa = [
    (fines_df['DELAY_TYPE'] == 'Early arrival') | (fines_df['DELAY_TYPE'] == '(0-30)mins'),
    (fines_df['DISTANCE_TYPE'] == 'Short') & (fines_df['DELAY_TYPE'] == '(30-60)mins'),
    (fines_df['DISTANCE_TYPE'] == 'Mid') & (fines_df['DELAY_TYPE'] == '(30-60)mins'),
    (fines_df['DISTANCE_TYPE'] == 'Long') & (fines_df['DELAY_TYPE'] == '(30-60)mins'),   
    
    (fines_df['DISTANCE_TYPE'] == 'Short') & (fines_df['DELAY_TYPE'] == '>1h'),
    (fines_df['DISTANCE_TYPE'] == 'Mid') & (fines_df['DELAY_TYPE'] == '>1h'),
    (fines_df['DISTANCE_TYPE'] == 'Long') & (fines_df['DELAY_TYPE'] == '>1h')]

# Añadimos la columna de multas
fines_df['FINE'] = np.select(condition_multa, choice_multa, default=int())

In [ ]:
# # Guardamos los datos preprocesados, para ser utilizados en la predicción posterior
# fines_df.to_parquet("dataFines.parquet", index=False)

## Distribución de las multas por aerolínea y retraso en función de la distancia

In [ ]:
multas = pd.DataFrame(fines_df.groupby(fines_df['AIRLINE'])["FLIGHT_NUMBER"].count())
multas = multas.rename(columns={"FLIGHT_NUMBER":"TOTAL_FLIGHTS"})
multas["SHORT_FLIGHTS"] =  fines_df[fines_df["DISTANCE_TYPE"]== "Short"].groupby(fines_df['AIRLINE'])["FLIGHT_NUMBER"].count()
multas["MID_FLIGHTS"] =  fines_df[fines_df["DISTANCE_TYPE"]== "Mid"].groupby(fines_df['AIRLINE'])["FLIGHT_NUMBER"].count()
multas["LONG_FLIGHTS"] =  fines_df[fines_df["DISTANCE_TYPE"]== "Long"].groupby(fines_df['AIRLINE'])["FLIGHT_NUMBER"].count()

multas['FINE'] = fines_df.groupby(fines_df['AIRLINE'])["FINE"].sum()

multas["FINE_SHORT"] =  fines_df[fines_df["DISTANCE_TYPE"]== "Short"].groupby(fines_df['AIRLINE'])["FINE"].sum()
multas["FINE_MID"] =  fines_df[fines_df["DISTANCE_TYPE"]== "Mid"].groupby(fines_df['AIRLINE'])["FINE"].sum()
multas["FINE_LONG"] =  fines_df[fines_df["DISTANCE_TYPE"]== "Long"].groupby(fines_df['AIRLINE'])["FINE"].sum()

# Una columna para repartir las multas en función del tiempo retrasado. Type I = (0-30mins) Type II = >1h
multas["FINE_SHORT_I"] =  fines_df[(fines_df['DISTANCE_TYPE'] == 'Short') & (fines_df['DELAY_TYPE'] == '(30-60)mins')].groupby(fines_df['AIRLINE'])["FINE"].sum()
multas["FINE_SHORT_II"] =  fines_df[(fines_df['DISTANCE_TYPE'] == 'Short') & (fines_df['DELAY_TYPE'] == '>1h')].groupby(fines_df['AIRLINE'])["FINE"].sum()
multas["FINE_MID_I"] =  fines_df[(fines_df['DISTANCE_TYPE'] == 'Mid') & (fines_df['DELAY_TYPE'] == '(30-60)mins')].groupby(fines_df['AIRLINE'])["FINE"].sum()
multas["FINE_MID_II"] =  fines_df[(fines_df['DISTANCE_TYPE'] == 'Mid') & (fines_df['DELAY_TYPE'] == '>1h')].groupby(fines_df['AIRLINE'])["FINE"].sum()
multas["FINE_LONG_I"] =  fines_df[(fines_df['DISTANCE_TYPE'] == 'Long') & (fines_df['DELAY_TYPE'] == '(30-60)mins')].groupby(fines_df['AIRLINE'])["FINE"].sum()
multas["FINE_LONG_II"] =  fines_df[(fines_df['DISTANCE_TYPE'] == 'Long') & (fines_df['DELAY_TYPE'] == '>1h')].groupby(fines_df['AIRLINE'])["FINE"].sum()

# Cuantos vuelos se han retrasado de cada tipo
multas["SHORT_DELAYED_I"] =  fines_df[(fines_df['DISTANCE_TYPE'] == 'Short') & (fines_df['DELAY_TYPE'] == '(30-60)mins')].groupby(fines_df['AIRLINE'])["FLIGHT_NUMBER"].count()
multas["SHORT_DELAYED_II"] =  fines_df[(fines_df['DISTANCE_TYPE'] == 'Short') & (fines_df['DELAY_TYPE'] == '>1h')].groupby(fines_df['AIRLINE'])["FLIGHT_NUMBER"].count()
multas["MID_DELAYED_I"] =  fines_df[(fines_df['DISTANCE_TYPE'] == 'Mid') & (fines_df['DELAY_TYPE'] == '(30-60)mins')].groupby(fines_df['AIRLINE'])["FLIGHT_NUMBER"].count()
multas["MID_DELAYED_II"] =  fines_df[(fines_df['DISTANCE_TYPE'] == 'Mid') & (fines_df['DELAY_TYPE'] == '>1h')].groupby(fines_df['AIRLINE'])["FLIGHT_NUMBER"].count()
multas["LONG_DELAYED_I"] =  fines_df[(fines_df['DISTANCE_TYPE'] == 'Long') & (fines_df['DELAY_TYPE'] == '(30-60)mins')].groupby(fines_df['AIRLINE'])["FLIGHT_NUMBER"].count()
multas["LONG_DELAYED_II"] =  fines_df[(fines_df['DISTANCE_TYPE'] == 'Long') & (fines_df['DELAY_TYPE'] == '>1h')].groupby(fines_df['AIRLINE'])["FLIGHT_NUMBER"].count()

multas = multas.fillna(0)
multas = multas.reset_index(level=0, drop=False)
# multas

In [ ]:
# Guardamos los datos preprocesados, para ser utilizados en la predicción posterior
# multas.to_parquet("dataFines.parquet", index=False)

In [ ]:
# multas.head()

## General fines distribution overview. Distance-Delay

In [ ]:
fig =go.Figure(go.Sunburst(
    
        labels=["Fines", "Short Flights", "S.(30-60 mins)","S.>1h",
                "Mid Flights", "M.(30-60 mins)","M.>1h",
                "Long Flights", "L.(30-60 mins)","L.>1h"],
        
        parents=["","Fines","Short Flights","Short Flights",
                    "Fines","Mid Flights","Mid Flights",
                    "Fines","Long Flights", "Long Flights"],

        values = [multas['FINE'].sum()]+[multas['FINE_SHORT'].sum()]+[multas['FINE_SHORT_I'].sum()]+[multas['FINE_SHORT_II'].sum()]+
             [multas['FINE_MID'].sum()]+[multas['FINE_MID_I'].sum()]+[multas['FINE_MID_II'].sum()]+
             [multas['FINE_LONG'].sum()]+[multas['FINE_LONG_I'].sum()]+[multas['FINE_MID_II'].sum()],

        #marker=dict(colors=px.colors.qualitative.Vivid)
        marker = dict(colors=["silver","paleturquoise","paleturquoise","paleturquoise",
                              "yellowgreen","yellowgreen","yellowgreen",
                              "mediumseagreen","mediumseagreen","mediumseagreen"]),
))

fig.update_layout(
    title="Fines distribution by flight distance and delay",
    template="plotly_dark",
    margin = dict(t=60, l=0, r=0, b=0),
    font_size=14,
    height = 650
)


fig.show()

## Flights distribution. Airline-Distance-Delay

In [ ]:
suma_delays = [multas['SHORT_DELAYED_I'].sum()]+[multas['SHORT_DELAYED_II'].sum()]+[multas['MID_DELAYED_I'].sum()]+[multas['MID_DELAYED_II'].sum()]+[multas['LONG_DELAYED_I'].sum()]+[multas['LONG_DELAYED_II'].sum()]

fig = go.Figure(data=[go.Sankey(
    node = dict(
      pad = 15,
      thickness = 20,
      line = dict(color = "black", width = 0.5),
      label = list(multas['AIRLINE'])+["Short", "Mid", "Long", "(30-60)mins", ">1h"],
            #   0     1     2         3       4        5             6        
      color = ["mediumseagreen"]*14+["yellowgreen"]*3+["paleturquoise"]*2
    ),
    link = dict(

      source =  [0,1,2,3,4,5,6,7,8,9,10,11,12,13]*3+[14,14,15,15,16,16],
        
             #  B1 B2 B2 C1 C1 C2
      target = list([14]*14+[15]*14+[16]*14+[17,18]*3),
      value  = list(multas['SHORT_FLIGHTS'])+list(multas['MID_FLIGHTS'])+list(multas['LONG_FLIGHTS'])+suma_delays,
      #color = ["mediumseagreen"]*14+["yellowgreen"]*3+["paleturquoise"]*2

  ))])

fig.update_layout(title_text="Flights distribution. Airline-Distance-Delay", font_size=10, template="plotly_dark", 
    height = 650)
fig.show()

## Fines distribution. Airline-Distance-Delay

In [ ]:
suma_fines = [multas['FINE_SHORT_I'].sum()]+[multas['FINE_SHORT_II'].sum()]+[multas['FINE_MID_I'].sum()]+[multas['FINE_MID_II'].sum()]+[multas['FINE_LONG_I'].sum()]+[multas['FINE_LONG_II'].sum()]

fig = go.Figure(data=[go.Sankey(
    node = dict(
      pad = 15,
      thickness = 20,
      line = dict(color = "black", width = 0.5),
      label = list(multas['AIRLINE'])+["Short", "Mid", "Long", "(30-60)mins", ">1h"],
            #   0     1     2         3       4        5             6        
      color = ["mediumseagreen"]*14+["yellowgreen"]*3+["paleturquoise"]*2
    ),
    link = dict(

      source =  [0,1,2,3,4,5,6,7,8,9,10,11,12,13]*3+[14,14,15,15,16,16],
        
             #  B1 B2 B2 C1 C1 C2
      target = list([14]*14+[15]*14+[16]*14+[17,18]*3),
      value  = list(multas['FINE_SHORT'])+list(multas['FINE_MID'])+list(multas['FINE_LONG'])+suma_fines

  ))])

fig.update_layout(title_text="Fines distribution. Airline-Distance-Delay", font_size=10, template="plotly_dark", 
    height = 650)
fig.show()

**NOTA:** G = mil millones de $

## Alternative visualization

In [ ]:
# fig = px.treemap(
#     fines_df[fines_df["DELAY_TYPE"].isin(["(30-60)mins",">1h"])], path= ["AIRLINE", "DISTANCE_TYPE", "DELAY_TYPE"], values='FINE',
#         color='ARRIVAL_DELAY', color_continuous_scale='Oryel')

# fig.update_layout(
#     title="Fines distribution. Airline-Distance-Delay",
#     template="plotly_dark",
#     margin = dict(t=50, l=25, r=25, b=25))
# fig.show()

Arrival delay = mean; Fine = sum